In [1]:
import numpy as np
import pandas as pd
import h2o
from h2o.automl import H2OAutoML
h2o.init(ip='localhost', nthreads=-1,min_mem_size='6G', max_mem_size='10G')

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_121"; OpenJDK Runtime Environment (Zulu 8.20.0.5-linux64) (build 1.8.0_121-b15); OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-linux64) (build 25.121-b15, mixed mode)
  Starting server from /home/vivekviswambharan/anaconda3/envs/3point6/h2o_jar/h2o.jar
  Ice root: /tmp/tmpoq53uzv0
  JVM stdout: /tmp/tmpoq53uzv0/h2o_vivekviswambharan_started_from_python.out
  JVM stderr: /tmp/tmpoq53uzv0/h2o_vivekviswambharan_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323... successful.


H2O cluster uptime:,17 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,9 months and 7 days !!!
H2O cluster name:,H2O_from_python_vivekviswambharan_9901t4
H2O cluster total nodes:,1
H2O cluster free memory:,8.89 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [2]:
train_df = pd.read_csv("final_train.csv")

In [ ]:
# unbalanced
train_df = pd.read_csv("train_df_wo.csv")

In [5]:
len(train_df)

5831426

In [7]:
train_df.columns

Index(['Discount', 'First_txn_days', 'Gender', 'Geo_Field', 'Income_Range',
       'Invoices', 'Item_Count', 'Job_Type', 'Language', 'Last_Pur_Week',
       'Last_Pur_Year', 'Last_accr_txn_days', 'Loyalty_Status',
       'Marital_Status', 'Nationality', 'Points', 'Region_Code',
       'Return_Reason', 'Revenue', 'Sales_Per_Day', 'State', 'Store_Format',
       'Store_Size_Sq_Ft', 'isMall', 'Prediction'],
      dtype='object')

In [7]:
test_df = pd.read_csv("test_df.csv")

In [6]:
len(test_df)

400000

In [ ]:
delete_index =train_df[train_df.Store_Code.isin(['60075', '60250', '60077', '60079' ])].index

In [ ]:
test_df = train_df.loc[delete_index, :]

In [ ]:
train_df.drop(delete_index, axis=0, inplace=True)

In [3]:
# dropping customer id and Store Code
# train_df.drop(['Customer_ID', 'Store_Code', 'Marital_Status', 'Income_Range', 'Job_Type', 'Points',\
#                'Loyalty_Status', 'Item_Count','Last_accr_txn_days', 'First_txn_days'],\
#                   axis=1, inplace=True)
train_df.drop(['Customer_ID', 'Store_Code'], axis=1, inplace=True)

In [7]:
# test_df.drop(['Customer_ID', 'Store_Code', 'Marital_Status', 'Income_Range', 'Job_Type', 'Points', 'Gender', 'Store_Format',\
#                'Loyalty_Status', 'Item_Count', 'Discount', 'Return_Reason', 'Last_Pur_Year'],\
#                   axis=1, inplace=True)
test_df.drop(['Customer_ID', 'Store_Code'], axis=1, inplace=True)

In [9]:
test_df.drop(['Prediction'], axis=1, inplace=True)

In [ ]:
# train_df.dtypes

In [10]:
for i in test_df.columns:
    test_df[i] = test_df[i].astype("float64")

In [ ]:
# test_df.dtypes

In [4]:
h2o_train = h2o.H2OFrame(train_df, destination_frame ="validTrain" )

/home/vivekviswambharan/anaconda3/envs/3point6/lib/python3.6/site-packages/h2o/utils/shared_utils.py:170: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


MemoryError: 

In [11]:
h2o_test = h2o.H2OFrame(test_df, destination_frame = "Test")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
len(test_df)/len(train_df)

In [ ]:
train_df["Prediction"]= train_df["Prediction"].asfactor()
test_df["Prediction"]= test_df["Prediction"].asfactor()

In [ ]:
# slice = int(len(df)*0.7)
train = train_df
test = test_df

y='Prediction'
x = train_df.columns
x = x.remove(y)

In [ ]:
aml = H2OAutoML(max_models = 2, seed = 1)
aml.train(x=x, y=y, training_frame=train, leaderboard_frame = test)

In [ ]:
lb = aml.leaderboard

In [ ]:
lb.head(rows=lb.nrows)

In [27]:
test_df = pd.read_excel("data/Test_Set.xlsx")

In [28]:
test_df.head()

,Customer_ID,Store_Code
0,1800000006365760,60298
1,1800000006365760,60299
2,1800000006365760,60300
3,1800000006365760,60301
4,1800000006365820,60298


In [29]:
test1_df = pd.read_csv("8baf6a23282e.csv")

In [30]:
test1_df.predict.value_counts()

0    256614
1    143386
Name: predict, dtype: int64

In [31]:
test_df["Prediction"] = test1_df.predict

In [32]:
test_df = pd.concat([test_df, test1_df], axis=1)

In [37]:
test_df.loc[test1_df.p1>0.4936, 'Prediction'] = 1
test_df.loc[test1_df.p1<=0.4936, 'Prediction'] = 0

In [38]:
final_df = test_df[["Customer_ID", "Store_Code", "Prediction"]]

In [39]:
final_df.head()

,Customer_ID,Store_Code,Prediction
0,1800000006365760,60298,1
1,1800000006365760,60299,0
2,1800000006365760,60300,1
3,1800000006365760,60301,1
4,1800000006365820,60298,0


In [40]:
final_df.Prediction.value_counts()

0    261054
1    138946
Name: Prediction, dtype: int64

In [41]:
final_df.to_csv("predicted.csv", index=False)

In [ ]:
test_df.shape

In [ ]:
test_df.Prediction.value_counts()